In [2]:
import numpy as np
from scipy.optimize import minimize
from numpy.linalg import inv

# Estimation of heat capacity using Bayes
np.random.seed(0)

# Generate data (true model is Cp = a0 + a1*T + a2*T^2)
a0 = 6.190
a1 = 0.02923
a2 = -0.0000007052
theta = np.array([a0, a1, a2])

# 3 sets of experiments at T=300K, T=600K, and T=900K
n = 20
N = 3 * n
x1 = np.linspace(300, 300, n)
x2 = np.linspace(600, 600, n)
x3 = np.linspace(900, 900, n)
x = np.concatenate((x1, x2, x3))

# Generate true outputs
y1 = a0 + a1*x1 + a2*x1**2
y2 = a0 + a1*x2 + a2*x2**2
y3 = a0 + a1*x3 + a2*x3**2
ytrue = np.concatenate((y1, y2, y3))

# Add random noise to true outputs
sigma = 0.1
y = ytrue + np.random.normal(0, sigma, N)

# Construct input and output data matrix
xsq = x**2
X = np.column_stack((np.ones(N), x, xsq))
Y = y

# Shuffle data
idxS = np.random.permutation(N)
X = X[idxS, :]
Y = Y[idxS]




In [4]:
# Formulate Bayesian estimation problem (full data)
Sigy = np.eye(N) * sigma**2
Sigp = np.eye(3)
thp = np.ones(3)

dat = {
    'Sigy': Sigy,
    'Sigp': Sigp,
    'thp': thp,
    'X': X,
    'Y': Y,
    'flag': 0  # ignores prior
}

# Define objective function
def myfun(th, dat):
    X = dat['X']
    Y = dat['Y']
    Sigy = dat['Sigy']
    Sigp = dat['Sigp']
    thp = dat['thp']
    flag = dat['flag']

    Yhat = X @ th
    like = 0.5 * ((Yhat - Y).T @ inv(Sigy) @ (Yhat - Y))
    prior = 0.5 * ((th - thp).T @ inv(Sigp) @ (th - thp))

    return like + flag * prior

# Solve problem
options = {'disp': False}
res = minimize(myfun, np.ones(3), args=(dat,), options=options)
thp = res.x

# Evaluate fit on full dataset
def msefun(theta, X, Y):
    Yhat = X @ theta
    return np.mean((Yhat - Y) ** 2)

MSE = msefun(thp, X, Y)

# Covariance of estimates
Sigp = inv(X.T @ inv(Sigy) @ X + inv(Sigp))

print(Sigp)

[[ 9.41059810e-03 -3.46706243e-05  2.75163684e-08]
 [-3.46706243e-05  1.34897621e-07 -1.10148023e-10]
 [ 2.75163684e-08 -1.10148023e-10  9.18282368e-14]]


In [5]:
# Now do recursively
S = 20  # batch size
Sigy = np.eye(S) * sigma**2
Sigp = np.eye(3)
thp = np.ones(3)

for k in range(N // S):
    dat['Sigy'] = Sigy
    dat['Sigp'] = Sigp
    dat['thp'] = thp
    dat['flag'] = 1

    in_idx = k * S
    out_idx = in_idx + S

    dat['X'] = X[in_idx:out_idx, :]
    dat['Y'] = Y[in_idx:out_idx]

    res = minimize(myfun, np.ones(3), args=(dat,), options=options)
    thp = res.x

    Sigp = inv(dat['X'].T @ inv(Sigy) @ dat['X'] + inv(Sigp))
    print(Sigp)

    MSE = msefun(thp, X, Y)

[[ 3.11083289e-02 -1.18170123e-04  9.43899948e-08]
 [-1.18170123e-04  4.71599141e-07 -3.85643689e-10]
 [ 9.43899948e-08 -3.85643689e-10  3.20389852e-13]]
[[ 1.48654251e-02 -5.50664847e-05  4.36434165e-08]
 [-5.50664847e-05  2.15155496e-07 -1.75148251e-10]
 [ 4.36434165e-08 -1.75148251e-10  1.45264959e-13]]
[[ 9.41059810e-03 -3.46706243e-05  2.75163684e-08]
 [-3.46706243e-05  1.34897621e-07 -1.10148023e-10]
 [ 2.75163684e-08 -1.10148023e-10  9.18282368e-14]]
